In [1]:
import aoc
import cmath
import collections
from collections import defaultdict
from functools import cache, reduce
import itertools as it
import math
import more_itertools as mit
import networkx as nx
import numpy as np
import operator as op
import pandas as pd
import re
import statistics
import utils

In [59]:
test_input="""00016C880162017C3686B18A3D4780"""
print(bin(int(test_input, 16)))

0b101101100100010000000000101100010000000010111110000110110100001101011000110001010001111010100011110000000


# Part 1

In [49]:
# run to solve part 1

def read_int(bits, index, amount):
    #print("read int:", index, amount, int(bits[index:index+amount], 2))
    return int(bits[index:index+amount], 2), index+amount

def read_bits(bits, index, amount):
    #print("read bits:", index, amount, bits[index:index+amount])
    return bits[index:index+amount], index+amount

def parse_packet(bits, index):
    start_index = index
    version, index = read_int(bits, index, 3)
    type_id, index = read_int(bits, index, 3)
    match type_id:
        case 4:
            # literal
            not_last = 1
            all_bits = ""
            while not_last:
                not_last, index = read_int(bits, index, 1)
                value, index = read_bits(bits, index, 4)
                all_bits += value
            value = int(all_bits, 2)
            print("literal:", index, version, type_id, value)
            return index, version, type_id, value
        case _:
            length_type_id, index = read_int(bits, index, 1)
            if length_type_id == 0:
                bits_remaining, index = read_int(bits, index, 15)
                print("bits_remaining:", index, version, type_id, length_type_id, bits_remaining)
                end_index = index + bits_remaining
                subpackets = []
                while index < end_index:
                    s = parse_packet(bits, index)
                    index = s[0]
                    subpackets.append(s)
                return index, version, type_id, length_type_id, subpackets
            else:
                subpackets_remaining, index = read_int(bits, index, 11)
                print("subpackets_remaining:", index, version, type_id, length_type_id, subpackets_remaining)
                subpackets = []
                for i in range(subpackets_remaining):
                    s = parse_packet(bits, index)
                    index = s[0]
                    subpackets.append(s)
                return index, version, type_id, length_type_id, subpackets

            
m = {
    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3' : '0011',
    '4' : '0100',
    '5' : '0101',
    '6' : '0110',
    '7' : '0111',
    '8' : '1000',
    '9' : '1001',
    'A' : '1010',
    'B' : '1011',
    'C' : '1100',
    'D' : '1101',
    'E' : '1110',
    'F' : '1111',
}
def part_1(aoc_input):
    bits ="".join(m[b] for b in aoc_input)
    #print(bits)
    result = parse_packet(bits, 0)
    packet_stack = [result]
    vers = []
    while packet_stack:
        cur_packet = packet_stack.pop()
        match cur_packet:
            case _, ver, 4, value:
                vers.append(ver)
            case _, ver, type_id, length_type_id, subpackets:
                packet_stack.extend(subpackets)
                vers.append(ver)
    return sum(vers)
                
                
    print(bits)

part_1_test_answer = part_1(test_input)
print("TEST RESULT: ", part_1_test_answer)
part_1_answer = part_1(aoc.get_input(2021, 16))
print("ACTUAL RESULT: ", part_1_answer)

bits_remaining: 22 5 0 0 91
subpackets_remaining: 40 1 0 1 1
subpackets_remaining: 58 3 0 1 5
literal: 69 7 4 6
literal: 80 6 4 6
literal: 91 5 4 12
literal: 102 2 4 15
literal: 113 2 4 15
TEST RESULT:  31
subpackets_remaining: 18 5 0 1 53
literal: 39 6 4 1523
bits_remaining: 61 3 1 0 413
subpackets_remaining: 79 2 3 1 1
subpackets_remaining: 97 3 2 1 1
bits_remaining: 119 1 1 0 355
bits_remaining: 141 7 0 0 333
subpackets_remaining: 159 0 1 1 1
subpackets_remaining: 177 2 2 1 1
subpackets_remaining: 195 7 2 1 1
bits_remaining: 217 0 3 0 257
subpackets_remaining: 235 4 1 1 1
bits_remaining: 257 3 2 0 217
bits_remaining: 279 4 1 0 195
bits_remaining: 301 1 3 0 173
subpackets_remaining: 319 6 3 1 1
subpackets_remaining: 337 3 3 1 1
bits_remaining: 359 4 3 0 115
bits_remaining: 381 5 2 0 93
bits_remaining: 403 5 3 0 71
subpackets_remaining: 421 3 3 1 1
bits_remaining: 443 5 2 0 31
literal: 474 0 4 131893
subpackets_remaining: 492 2 0 1 5
literal: 513 0 4 4006
literal: 534 0 4 1033
literal

In [50]:
# run to submit part 1

aoc.submit_answer(2021, 16, 1, part_1_answer)

Submitting for YEAR 2021, DAY 16, LEVEL 1: 891... Correct!


# Part 2

In [53]:
# run to solve part 2

test_input="""04005AC33890"""

def value_of_packet(packet):
    match packet:
        case _, _, 4, value:
            return value
        case _, _, type_id, _, subpackets:
            match type_id:
                case 0:
                    return sum(value_of_packet(p) for p in subpackets)
                case 1:
                    return math.prod(value_of_packet(p) for p in subpackets)
                case 2:
                    return min(value_of_packet(p) for p in subpackets)
                case 3:
                    return max(value_of_packet(p) for p in subpackets)
                case 5:
                    p1 = value_of_packet(subpackets[0])
                    p2 = value_of_packet(subpackets[1])
                    return 1 if p1 > p2 else 0
                case 6:
                    p1 = value_of_packet(subpackets[0])
                    p2 = value_of_packet(subpackets[1])
                    return 1 if p1 < p2 else 0
                case 7:
                    p1 = value_of_packet(subpackets[0])
                    p2 = value_of_packet(subpackets[1])
                    return 1 if p1 == p2 else 0

def part_2(aoc_input):
    bits ="".join(m[b] for b in aoc_input)
    #print(bits)
    result = parse_packet(bits, 0)
    return value_of_packet(result)

part_2_test_answer = part_2(test_input)
print("TEST RESULT: ", part_2_test_answer)
part_2_answer = part_2(aoc.get_input(2021, 16))
print("ACTUAL RESULT: ", part_2_answer)

bits_remaining: 22 0 1 0 22
literal: 33 5 4 6
literal: 44 3 4 9
TEST RESULT:  54
subpackets_remaining: 18 5 0 1 53
literal: 39 6 4 1523
bits_remaining: 61 3 1 0 413
subpackets_remaining: 79 2 3 1 1
subpackets_remaining: 97 3 2 1 1
bits_remaining: 119 1 1 0 355
bits_remaining: 141 7 0 0 333
subpackets_remaining: 159 0 1 1 1
subpackets_remaining: 177 2 2 1 1
subpackets_remaining: 195 7 2 1 1
bits_remaining: 217 0 3 0 257
subpackets_remaining: 235 4 1 1 1
bits_remaining: 257 3 2 0 217
bits_remaining: 279 4 1 0 195
bits_remaining: 301 1 3 0 173
subpackets_remaining: 319 6 3 1 1
subpackets_remaining: 337 3 3 1 1
bits_remaining: 359 4 3 0 115
bits_remaining: 381 5 2 0 93
bits_remaining: 403 5 3 0 71
subpackets_remaining: 421 3 3 1 1
bits_remaining: 443 5 2 0 31
literal: 474 0 4 131893
subpackets_remaining: 492 2 0 1 5
literal: 513 0 4 4006
literal: 534 0 4 1033
literal: 555 4 4 540
literal: 566 2 4 1
literal: 592 2 4 35584
subpackets_remaining: 610 7 1 1 2
literal: 636 1 4 56324
subpackets_r

In [54]:
# run to submit part 2

aoc.submit_answer(2021, 16, 2, part_2_answer)

Submitting for YEAR 2021, DAY 16, LEVEL 2: 673042777597... Correct!
